<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/Vision_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [ ]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wandb
from wandb.keras import WandbCallback
import PIL
import PIL.Image
import sys
from google.colab import files

In [ ]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

In [ ]:
wandb.init(project='your_project_name')

data_dir = '/path/to/your/faulty_directory'
img_height, img_width = 240, 380

# Data generator with validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

# Training and validation data generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define the Vision Transformer model
def create_vit_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Reshape((-1, x.shape[-1]))(x)  # Reshape for transformer

    # Transformer block
    for _ in range(4):  # 4 transformer blocks
        x_res = x
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
        x = layers.Add()([x_res, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x_res = x
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dense(x.shape[-1])(x)
        x = layers.Add()([x_res, x])

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_vit_model((img_height, img_width, 3))
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',  # Save the best model in Keras format
    monitor='val_loss',  # Metric to monitor
    save_best_only=True,  # Save only the best model
    mode='min',  # Minimize the validation loss
    verbose=1  # Verbose output for logging
)

# Train the model with W&B callback
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint, WandbCallback()]  # Add W&B callback
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}, Accuracy: {accuracy}')